In [1]:
import pandas as pd
import numpy as np
orders = pd.read_csv("orders.csv")
items = pd.read_csv("Items_orders.csv")
itemProps = pd.read_csv("Items_attribute.csv",encoding='gbk')
orders_items = pd.merge(orders,items,on="订单编号")
orders_items_props = pd.merge(orders_items,itemProps,on="标题")

In [2]:
result = orders_items_props.loc[:,["买家会员名","宝贝ID"]]
result["购买次数"] = 0
freq = result.groupby(["买家会员名","宝贝ID"]).count().reset_index()
#freq["购买次数"]=1
freq= freq.pivot(index="买家会员名",columns="宝贝ID",values="购买次数")
freqMatrix = freq.fillna(0).values

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
user_similar=cosine_similarity(freqMatrix)

In [4]:
#构建预测函数
def predict(similar):
    user_cnt = freqMatrix.shape[0]#用户数3318
    item_cnt = freqMatrix.shape[1]#商品数269
    pred = np.zeros((user_cnt,item_cnt))
    for uid in range(user_cnt):
        for iid in range(item_cnt):
            if freqMatrix[uid,iid] == 0:
                #print (uid,iid)
                pred[uid,iid] = Recommendation_s(uid,iid,similar)
    return pred

#构建一个基于用户和物品的推荐
def Recommendation_s(uid,iid,similar,k=10):
    score = 0
    weight = 0
    uid_action = freqMatrix[uid,:]      #用户uid 对所有商品的行为评分  
    iid_action = freqMatrix[:,iid]      #物品iid 得到的所有用户评分  

    uid_sim = similar[uid,:]      #用户uid 对所有用户的相似度    
    sim_indexs = np.argsort(uid_sim)[-(k+1):-1]  #最相似的k个用户的index（除了自己）
    uid_i_mean = np.sum(uid_action)/uid_action[uid_action!=0].size#
    for j in sim_indexs :
        if iid_action[j]!=0:
            uid_j_action = freqMatrix[j,:]
            uid_j_mean = np.sum(uid_j_action)/uid_j_action[uid_j_action!=0].size
            score += uid_sim[j]*(iid_action[j]-uid_j_mean)
            weight += abs(uid_sim[j])

    if weight==0:  
        return 0
    else:
        return uid_i_mean + score/float(weight)

In [5]:
def get_recom(prediction,n=5):
    recom_df = pd.DataFrame(prediction,columns=freq.columns,index=freq.index)
    
    recom_df = recom_df.stack().reset_index()
    recom_df.rename(columns={0:"推荐指数"},inplace=True)
    
    grouped = recom_df.groupby("买家会员名")
   
    topk = grouped.apply(get_topk,n=n)
    
    topk = topk.drop(["买家会员名"],axis=1)
    
    topk.index = topk.index.droplevel(1)
   
    topk.reset_index(inplace=True)
#     print(topk)
    return topk
    
def get_topk(group,n):
    return group.sort_values("推荐指数",ascending=False)[:n]

In [6]:
user_prediction = predict(user_similar)

In [7]:
user_recom = get_recom(user_prediction,3)
user_recom

,买家会员名,宝贝ID,推荐指数
0,0,527419046969,0.500000
1,0,538658965256,0.428571
2,0,542939108885,0.428571
3,1,544066720474,1.000000
4,1,35721027449,0.666667
5,1,36074765406,0.666667
6,2,35721027449,0.000000
7,2,544114479515,0.000000
8,2,543695068744,0.000000
9,3,35721027449,0.000000
